In [1]:
from functools import reduce
from json import loads

import pandas as pd
import requests
from fake_useragent import UserAgent

In [12]:
ua = UserAgent()
url = "https://web.archive.org/web/20221206170529/https://cdn.cloudflare.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=12-06-2022-17"
headers = {"User-Agent": ua.random}

response = requests.get(url, headers=headers)
startidx = response.text.find("(")
endidx = response.text.find(")")
data = loads(response.text[startidx + 1 : endidx])
series_list = loads(data["json"])

In [13]:
df_list = []
for series in series_list:
    df_dict = {}
    region = series["label"]
    df_dict["Timestamp"] = [
        pd.to_datetime(x[0], unit="ms") for x in series["data"]
    ]
    df_dict[region] = [int(x[1]) for x in series["data"]]
    df_list.append(pd.DataFrame(df_dict))

df = reduce(
    lambda x, y: pd.merge(x, y, on="Timestamp", how="outer"), df_list
)
df = df.sort_values("Timestamp").reset_index(drop=True)

In [14]:
df

,Timestamp,Central America,Africa,Middle East,Oceania,South America,Russia,Asia,Europe,North America
0,2022-12-04 16:50:00,38,64,471,79,1005,1002,3842,4460,2451
1,2022-12-04 17:00:00,35,64,471,79,1005,1002,3637,4414,2451
2,2022-12-04 17:10:00,38,64,475,79,1007,1002,3637,4460,2522
3,2022-12-04 17:20:00,38,64,482,79,1049,1002,3530,4585,2617
4,2022-12-04 17:30:00,38,63,483,76,1093,987,3369,4629,2679
...,...,...,...,...,...,...,...,...,...,...
283,2022-12-06 16:00:00,34,79,577,144,955,1248,8495,4814,2197
284,2022-12-06 16:10:00,36,79,577,145,965,1274,8465,5033,2355
285,2022-12-06 16:20:00,36,80,579,145,994,1330,8227,5238,2515
286,2022-12-06 16:30:00,36,85,579,142,999,1348,7720,5333,2544


In [75]:
df.loc[df["Timestamp"] == pd.Timestamp("2024-11-05 23:00:00")]

,Timestamp,Central America,Africa,Middle East,Oceania,South America,Russia,Asia,Europe,North America
